### Import Modules, Change Directory, Set Variables

In [13]:
output_location = '/Users/johnmccloskey/Desktop'
output_name = 'output.tsv'
assessment_n = 1 # assessment through which you want all features

In [2]:
import os
import re
from zipfile import ZipFile
import pandas as pd
import json
os.chdir('..') # Please comment out after done running this cell. If you go too far up, use the next line.
# os.chdir('opengamedata')
os.getcwd()
from game_info.Jowilder import Jowilder_Enumerators as je

### Look to see what proc_zips are available, choose one to open

In [3]:
data_dir = 'data/JOWILDER'
jo_wilder_pattern = re.compile('JOWILDER_([0-9]+)_to_([0-9+])*')
files = [x for x in sorted(os.listdir(data_dir),reverse=True) if 'proc.zip' in x]
get_file_by_n = lambda n: os.path.join(data_dir, files[n])
[f'{i}: {f}' for i,f in enumerate(files)][:10]

['0: JOWILDER_20191201_to_20191213_0e39a60_proc.zip',
 '1: JOWILDER_20191201_to_20191211_dc8553c_proc.zip',
 '2: JOWILDER_20191201_to_20191210_d333c49_proc.zip',
 '3: JOWILDER_20191201_to_20191206_1bcf69b_proc.zip',
 '4: JOWILDER_20191201_to_20191206_160cf79_proc.zip',
 '5: JOWILDER_20191201_to_20191204_138f261_proc.zip',
 '6: JOWILDER_20191201_to_20191203_9700a60_proc.zip',
 '7: JOWILDER_20191101_to_20191126_e8227f2_proc.zip',
 '8: JOWILDER_20191101_to_20191124_e8227f2_proc.zip',
 '9: JOWILDER_20191101_to_20191101_e8227f2_proc.zip']

In [4]:
read_file = 0 # CHANGE: This is the enum of the file you wish to open

### Import to dataframe, verify roughly that is it what is expected

In [5]:
zf = ZipFile(get_file_by_n(read_file))
with zf.open(zf.namelist()[0]) as f:
    df = pd.read_csv(f,index_col='sessionID')
df.head()

/Users/johnmccloskey/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (275,291,294,309,310) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,lvl0_EventCount,lvl1_EventCount,lvl2_EventCount,lvl3_EventCount,lvl4_EventCount,lvl5_EventCount,lvl6_EventCount,lvl7_EventCount,lvl8_EventCount,lvl9_EventCount,...,sess_count_hovers,sess_meaningful_action_count,sess_EventCount,sess_timeToRead,sess_avgTimePerWord,sess_avgTimePerTextBox,sess_stDevTimePerTextBox,sess_words_per_second,sess_count_notebook_uses,sess_avg_time_between_clicks
sessionID,,,,,,,,,,,,,,,,,,,,,
19110007435189656,38,25.0,79.0,33.0,58.0,14.0,148.0,66.0,41.0,49.0,...,327,150,1356,0,0,2.617716,0,11.738549,0,1.428273
19110008462915572,34,27.0,54.0,50.0,64.0,17.0,97.0,94.0,53.0,45.0,...,201,74,734,0,0,5.943833,0,5.567715,0,1.516161
19110009443518108,45,59.0,49.0,69.0,971.0,26.0,207.0,195.0,NaN,NaN,...,0,169,1621,0,0,0.880345,0,23.207069,0,0.400679
19110011242566930,48,44.0,58.0,137.0,92.0,36.0,126.0,177.0,88.0,130.0,...,855,203,2449,0,0,5.229271,0,5.842647,0,1.566259
19110011385682280,34,44.0,40.0,51.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,31,169,0,0,4.904818,0,10.551951,0,0.942705


In [6]:
df.shape

(2275, 381)

### Import Schema, look at available features

In [7]:
schema_path = 'schemas/JSON/JOWILDER.json'
with open(schema_path) as f:
    schema = json.load(f)
features = {k:list(v.keys()) for k,v in schema['features'].items()}
lvl_prefix = lambda n: f'lvl{n}_'
q_prefix = lambda n: f'Q{n}_'

In [8]:
print(*features.items(),sep='\n\n')

('perlevel', ['EventCount', 'count_clicks', 'count_hovers', 'avgTimePerTextBox', 'time_in_level', 'meaningful_action_count', 'words_per_second', 'count_notebook_uses', 'avg_time_between_clicks'])

('per_custom_count', ['num_guesses', 'answers', 'A1', 'A2', 'A3', 'A1_time', 'A2_time', 'A3_time'])

('aggregate', ['sessionID', 'persistentSessionID', 'sessDuration', 'sess_count_clicks', 'sess_count_hovers', 'sess_meaningful_action_count', 'sess_EventCount', 'sess_timeToRead', 'sess_avgTimePerWord', 'sess_avgTimePerTextBox', 'sess_stDevTimePerTextBox', 'sess_words_per_second', 'sess_count_notebook_uses', 'sess_avg_time_between_clicks'])


### Drop columns past given assessment, drop rows that did not answer the last question

In [9]:
qs_through_assessement_n = lambda n: list(range(je.assessment_to_last_q[n]+1))
lvls_through_assessment_n = lambda n: list(range(je.assessement_to_lvl[n]+1))
def features_through_assessment_n(n):
    ret = []
    for lvl in lvls_through_assessment_n(n):
        for f in features['perlevel']:
            ret.append(lvl_prefix(lvl)+f)
    for q in qs_through_assessement_n(n):
        for f in features['per_custom_count']:
            ret.append(q_prefix(q)+f)
    ret.extend([f for f in features['aggregate'] if f != 'sessionID'])
    return ret
assessment_1_df = df[features_through_assessment_n(assessment_n)]
assessment_1_df = assessment_1_df.dropna(subset=[f'Q{je.assessment_to_last_q[assessment_n]}_A1'])

In [10]:
assessment_1_df.shape

(576, 90)

In [11]:
assessment_1_df.head()

,lvl0_EventCount,lvl0_count_clicks,lvl0_count_hovers,lvl0_avgTimePerTextBox,lvl0_time_in_level,lvl0_meaningful_action_count,lvl0_words_per_second,lvl0_count_notebook_uses,lvl0_avg_time_between_clicks,lvl1_EventCount,...,sess_count_hovers,sess_meaningful_action_count,sess_EventCount,sess_timeToRead,sess_avgTimePerWord,sess_avgTimePerTextBox,sess_stDevTimePerTextBox,sess_words_per_second,sess_count_notebook_uses,sess_avg_time_between_clicks
sessionID,,,,,,,,,,,,,,,,,,,,,
19110013291215470,2,NaN,NaN,NaN,0.002,NaN,NaN,NaN,NaN,NaN,...,111,38,390,0,0,6.620167,0,5.666645,0,3.550675
19110013515125732,34,28.0,4.0,3.597400,49.450,2.0,3.372873,NaN,1.640185,32.0,...,182,82,664,0,0,3.837429,0,4.911063,0,1.613556
19110014215928624,24,19.0,3.0,2.158000,25.215,1.0,5.841730,NaN,1.288889,34.0,...,263,114,1091,0,0,5.392845,0,7.409123,0,1.240797
19110014312317684,88,72.0,14.0,3.037444,139.889,7.0,3.117215,NaN,1.641817,33.0,...,245,72,953,0,0,14.092947,0,2.048160,0,2.489691
19110016345644932,40,28.0,10.0,1.154714,200.637,4.0,5.354636,NaN,1.410185,34.0,...,522,153,1714,0,0,5.708855,0,6.087589,0,2.864502


### Export

In [12]:
### Import to dataframe, verify roughly that is it what is expected
assessment_1_df.to_csv(os.path.join(output_location,output_name), sep='\t')